# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [100]:
# your code here
orders = pd.read_csv('Orders.zip') #otherwise, if multiple files inside: import zipfile
orders.head()

#orders.describe()
#most expensive orders:
#set(orders.nlargest(5, "amount_spent")["amount_spent"])
#orders[orders["amount_spent"].isin(set(orders.nlargest(5, "amount_spent")["amount_spent"]))]

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [102]:
# aggregate by the spendings:
customer_spendings = orders.groupby("CustomerID").agg({"amount_spent": "sum"}).reset_index()
customer_spendings
#customer_spendings.describe()

#calculate the quantiles needed
quantile95 = np.percentile(customer_spendings["amount_spent"], 95) #5840.181999999982
quantile75 = np.percentile(customer_spendings["amount_spent"], 75) #1661.64

#get customer number only in the quantile areas requested, save them as a set
customers_vip = customer_spendings[customer_spendings["amount_spent"]>quantile95]["CustomerID"]
customers_vip = set(customers_vip)

customers_preferred = customer_spendings[(customer_spendings["amount_spent"]>quantile75) & (customer_spendings["amount_spent"]<quantile95)]["CustomerID"]
customers_preferred = set(customers_preferred)

#assign to a new column:
#VIP if the customer is in VIP-set, preferred if in preferred-set, otherwise False
orders["customer_group"] = np.where(orders["CustomerID"].isin(customers_vip), "VIP",np.where(orders["CustomerID"].isin(customers_preferred), "preferred", False))
#test it:
#orders[orders["customer_group"]=="VIP"]

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [98]:
# your code here
orders.head()

#group by country and filter for customer_group
orders_vip = orders[orders["customer_group"]=="VIP"]
orders_vip.groupby("Country").agg({"CustomerID" : "count"}).sort_values("CustomerID", ascending=False)

#solution:
#The UK has the most VIP Customers, in total 84.185

,CustomerID
Country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080
Australia,898
Portugal,681
Switzerland,594
Spain,511


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [107]:
#group by country and filter for customer_group VIP and preferred
orders_vip_pref = orders[(orders["customer_group"]=="VIP") | (orders["customer_group"]=="preferred")]

orders_vip_pref.groupby("Country").agg({"CustomerID" : "count"}).sort_values("CustomerID", ascending=False)

#solution:
#The UK has the most VIP+preferred Customers combined, in total 221.635

,CustomerID
Country,
United Kingdom,221635
Germany,7349
EIRE,7238
France,6301
Netherlands,2080
Spain,1569
Belgium,1557
Switzerland,1370
Portugal,1093
